In [2]:
import pandas as pd


fluxes_3x3 = pd.read_csv("output_RMM_3000_3000_1500/3000_3000_1500_rmm_flux.csv")
xs = pd.read_csv("output_RMM_3000_3000_1500/3000_3000_1500_rmm_xs.csv")


In [3]:
pins_fuel = {
  1: {"n_id": [1,2,6,7], "reg_id": [65,68,83,88]},
  2: {"n_id": [3,4,8,9], "reg_id": [71,75,93,97]},
  3: {"n_id": [5,6,11,12], "reg_id": [79,82,105,108]},
  4: {"n_id": [7,8,13,14], "reg_id": [86,91,114,119]},
  5: {"n_id": [9,10,15,16], "reg_id": [96,101,122,127]},
  6: {"n_id": [12,13,17,18], "reg_id": [109,114,131,134]},
  7: {"n_id": [14,15,19,20], "reg_id": [119,123,137,141]},
}
energy_groups = 8
fuel_fluxes = fluxes_3x3[fluxes_3x3["material"] == "inner_fuel"]

pins_fluxes = { }

for g in range(energy_groups):
  pins_fluxes[g+1] = {}
  for p_id, node_ids in pins_fuel.items():
    pins_fluxes[g+1][p_id] = fluxes_3x3[
      (fluxes_3x3["region_id"].isin(pins_fuel[p_id]["reg_id"])) & 
      (fluxes_3x3["Energy_group"] == g+1) &
      (fluxes_3x3["material"] == "inner_fuel")
    ]


In [13]:
import numpy as np

pin_power = []
for g in range(energy_groups):
  pin_power_g = []
  for p_id in pins_fuel.keys():
    f_data = pins_fluxes[g+1][p_id]
    sum_pin_power = 0.0
    for reg in pins_fuel[p_id]["reg_id"]:
      flx = f_data[(f_data["region_id"] == reg) & (f_data["Energy_group"] == g+1)]["scalar_flux"]
      vol = f_data[(f_data["region_id"] == reg) & (f_data["Energy_group"] == g+1)]["volume"]
      xs_fiss = xs[(xs["region_id"] == reg) & (xs["Energy_group"] == g+1)]["xs_fiss"]
      sum_pin_power += float(flx * vol * xs_fiss)
    pin_power_g.append(sum_pin_power / 0.40039284210736553)
  pin_power.append(pin_power_g)

pin_power = np.array(pin_power)
# print(pin_power)
for g in range(pin_power.shape[0]):
  pin_power[g] = pin_power[g] / pin_power[g].max()

# pin_power /= pin_power.max()
print(pin_power)




[[0.97600006 0.97600006 1.         0.98654329 1.         0.97600006
  0.97600006]
 [0.9742227  0.9742227  1.         0.98188728 1.         0.9742227
  0.9742227 ]
 [0.97157154 0.97157154 1.         0.98455562 1.         0.97157154
  0.97157154]
 [0.972709   0.972709   1.         0.98397157 1.         0.972709
  0.972709  ]
 [0.97218406 0.97218406 1.         0.98385004 1.         0.97218406
  0.97218406]
 [0.97132832 0.97132832 1.         0.98273864 1.         0.97132832
  0.97132832]
 [0.96298262 0.96298262 1.         0.95139455 1.         0.96298262
  0.96298262]
 [0.95754597 0.95754597 1.         0.9476147  1.         0.95754597
  0.95754597]]
